# Wikipedia＋my-ope文書を使用しDoc2Vecモデルを生成

## (1) Wikipediaコンテンツファイルから全文書を抽出

レポート <a href="31-Wikipedia-contents-csv.ipynb"><b>31-Wikipedia-contents-csv.ipynb</b></a> の手順にて、いったんローカルPCにCSVファイル化しておきます。

## (2) Wikipedia文書のTaggedDocumentを生成

Wikipedia文書のCSVファイルからデータフレームを読み込み、TaggedDocumentに変換します。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [3]:
from gensim.models.doc2vec import TaggedDocument
#from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_ids, wiki_contents):
    '''
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    for index, _ in enumerate(wiki_ids):
        tag = wiki_ids[index]

        sentences = wiki_contents[index]
        separated_sentences = Nlang_naive.split(sentences)
        words = separated_sentences.split()

        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [4]:
'''
    Wikipedia文書から生成されたCSVを読み込み、
    配列を生成する
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'
csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df = pd.read_csv(csv_file_name)

In [5]:
_wiki_ids = np.array(df['id'])
_wiki_titles = np.array(df['title'])
_wiki_contents = np.array(df['content'])

In [6]:
'''
    時間がかかるので一部のみ使用します（全量＝100万件）
'''
partial_cnt = 10000

def split_wiki_corpus(wiki_id, wiki_content, wiki_title):
    '''
        コーパスを分割

        予測処理のインプットとなる単語数に合わせるため、
        句読点でコーパスを分割する。
        ただし、この段階でIDをユニーク化すると
        分割されたコーパス間で意味が繋がらなくなるため、
        IDを [doc id]_[serial] 形式で付番する
    '''
    sub_ids = []
    sub_contents = []
    titles = []
    
    sub_index = 0
    for sub_content in wiki_content.split('。'):
        if len(sub_content) == 0:
            continue
        if sub_content[0] == ' ':
            continue

        sub_id = '%d_%d' % (wiki_id, sub_index)
        sub_ids.append(sub_id)
        sub_contents.append(sub_content)
        titles.append(wiki_title)
        sub_index += 1
        
        if sub_index == 10:
            break

    return sub_ids, sub_contents, titles

wiki_ids = []
wiki_contents = []
wiki_titles = []

for index in range(partial_cnt):
    wiki_id = _wiki_ids[index]
    wiki_content = _wiki_contents[index]
    wiki_title = _wiki_titles[index]

    sub_ids, sub_contents, titles = split_wiki_corpus(wiki_id, wiki_content, wiki_title)

    wiki_ids.extend(sub_ids)
    wiki_contents.extend(sub_contents)
    wiki_titles.extend(titles)

In [7]:
wiki_tagged_document_list = get_tagged_document_list(wiki_ids, wiki_contents)

In [8]:
len(wiki_tagged_document_list)

87031

In [9]:
len(wiki_titles)

87031

## (3) my-ope文書のTaggedDocumentを生成

In [10]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 13 # toyotsu_human.csv
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [11]:
'''
    マイオペの学習セットを取得
'''
_datasource = Datasource(type='csv')
#learning_training_messages = _datasource.learning_training_messages(_bot_id)
learning_training_messages = _datasource.question_answers(_bot_id)

questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])
ids = np.array(learning_training_messages['id'])

2017/06/01 AM 11:08:51 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/06/01 AM 11:08:51 ['./fixtures/question_answers/toyotsu_human.csv']


In [12]:
len(questions)

317

In [13]:
'''
    既存セットの否定の意味を持つトレーニングセットを追加します。
    例）
    　'「退職金」退職金の振込口座を変えたい（正解＝6975）'
    　に対して、否定の意味を持つセット
    　'「退職金」退職金の振込口座を変えたくない'
'''
questions_nega = np.array(['「退職金」退職金の振込口座を変えたくない'])
questions = np.append(questions, questions_nega)

In [14]:
answer_ids_nega = np.array([7050])
answer_ids = np.append(answer_ids, answer_ids_nega)

In [15]:
ids_nega = np.array([13700])
ids = np.append(ids, ids_nega)

In [16]:
'''
    正しく要素が追加されたかどうかチェックします。
'''
print('count: questions=%d, answer_ids=%d (class count=%d), ids=%d' % (
    np.size(questions), np.size(answer_ids), np.size(np.unique(answer_ids)), np.size(ids)
))

count: questions=318, answer_ids=318 (class count=303), ids=318


In [17]:
'''
    マイオペの学習セットから、コーパスを生成
    
    タグとして、learning_training_messages.id を付与
'''
myope_tagged_document_list = get_tagged_document_list(ids, questions)
len(myope_tagged_document_list)

318

## (4) 両方のTaggedDocumentを合体

In [18]:
all_tagged_document_list = []
all_tagged_document_list.extend(wiki_tagged_document_list)
all_tagged_document_list.extend(myope_tagged_document_list)

In [19]:
len(all_tagged_document_list)

87349

## (5) 学習実行とモデルファイル作成

In [20]:
from gensim.models.doc2vec import Doc2Vec
import time

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wiki_myope.PV%d.model' % dm
    return model_path

def train(tagged_document_list, dm, size):
    '''
        tagged_document_listは、
        Wikipedia文書のタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成と学習を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        学習が完了したら、モデルをファイルに保存します。
    '''
    start = time.time()
    model = Doc2Vec(documents=tagged_document_list, dm=dm, size=size, 
                    window=20, min_count=1, iter=size*10, workers=100)
    elapsed_time =  time.time() - start
    if dm:
        s = 'DM'
    else:
        s = 'DBoW'
    print("Doc2Vec: trained by %s: elapsed %d seconds" % (s, elapsed_time))
    print('Doc2Vec: document vector size=%d, feature size=%d' % (len(model.docvecs), size))

    model.save(doc2vec_model_path(dm))

    return model

In [21]:
'''
    tagged_document_listを使用し、学習実行（PV-DBOW）
'''
model_dbow = train(all_tagged_document_list, dm=0, size=200)

Doc2Vec: trained by DBoW: elapsed 6317 seconds
Doc2Vec: document vector size=87349, feature size=200


In [22]:
#'''
#    tagged_document_listを使用し、学習実行（PV-DM）
#'''
#model_dm = train(all_tagged_document_list, dm=1, size=200)